# Experiments

## I. Imports & functions

In [289]:
# required imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from tslearn.clustering import TimeSeriesKMeans
from dtaidistance import dtw_ndim
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics.pairwise import euclidean_distances
import skfda

# using R inside python
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
import rpy2.robjects.pandas2ri

rpy2.robjects.numpy2ri.activate()
rpy2.robjects.pandas2ri.activate()

# install R packages
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

# run if not installed previously from requirements.txt
# utils.install_packages('clValid')
# utils.install_packages('symbolicDA')

# load R packages
clValid = importr('clValid')
symbolicDA = importr('symbolicDA')
stats = importr('stats')

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [290]:
# functions
def kmeans_clustering(data: pd.DataFrame, n_clusters: int, metric: str) -> TimeSeriesKMeans:
    """
    Perform KMeans clustering.

    Args:
        data (pd.DataFrame): preprocessed dataframe with economic indexes
        n_clusters (int): number of clusters to be formed

    Returns:
        TimeSeriesKMeans: fitted clustering model
    """
    # transform input data into adequate structure - 3D numpy array
    data_agg = data.drop('year', axis=1).groupby(['countrycode', 'country']).agg(list)
    n_countries = data_agg.shape[0] # number of points (countries)
    time_range =  len(data['year'].drop_duplicates()) # time range
    n_vars = data.shape[1] - 3 # number of economic indexes
    # filling the array
    data_agg_arr = np.empty(shape=(n_countries, n_vars, time_range))
    for i in range(data_agg.shape[0]):
        for j in range(data_agg.shape[1]):
            data_agg_arr[i][j] = np.array(data_agg.iloc[i,j])
    # creating and fitting a model
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric=metric)
    model.fit(data_agg_arr)
    return model

def agglomerative_clustering(matrix: np.matrix, n_clusters: int, linkage: str) -> AgglomerativeClustering:
    """
    Perform hierarchical clustering.

    Args:
        data (pd.DataFrame): preprocessed dataframe with economic indexes
        n_clusters (int): number of clusters to be formed
        linkage (str): type of linkage criterion; 'average', 'complete' or 'single'

    Returns:
        AgglomerativeClustering: fitted clustering model
    """
    # creating and fitting the model
    model = AgglomerativeClustering(
        n_clusters=n_clusters, affinity='precomputed', linkage=linkage, compute_distances=True)
    model.fit(matrix)
    return model

def dbscan_clustering(matrix: np.matrix, eps: float, min_samples: int) -> DBSCAN:
    """
    Perform DBSCAN clustering.

    Args:
        data (pd.DataFrame): preprocessed dataframe with economic indexes
        eps (float): maximum distance between two points for them to be considered as neighbouring
        min_samples (int): number of samples in a neighborhood for a point to be considered as a core point

    Returns:
        DBSCAN: fitted clustering model
    """
    # creating and fitting the model
    model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
    model.fit(matrix)
    return model


In [291]:
def plot_clustering(countries: pd.DataFrame, labels: np.array) -> None:
    """
    Plot cartogram presenting clustering results for given countries.

    Args:
        countries (pd.DataFrame): Pandas Dataframe containing at least one column, named 'countrycode', 
        with ISO-3166 alpha-3 codes of countries
        labels (np.array): cluster assignment generated by clustering model for given countries
    """
    labels = labels.astype(str)
    countries["cluster"] = pd.Series(labels)
    fig = px.choropleth(countries, locations = 'countrycode', color = "cluster", 
                        projection='conic conformal', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_geos(lataxis_range=[35, 75], lonaxis_range=[-15, 45]) # customized to show Europe only
    fig.show()
    return fig

def evaluate_clustering(matrix: np.matrix, kmeans: np.array, agg: np.array, dbscan: np.array) -> pd.DataFrame:
    """
    Compare 3 used algoritms using following metrics: silhouette score (more to be defined).

    Args:
        data (pd.DataFrame): preprocessed dataframe with economic indexes
        kmeans (np.array): cluster assignment generated by KMeans clustering
        agg (np.array): cluster assignment generated by hierarchical clustering
        dbscan (np.array): cluster assignment generated by DBSCAN clustering

    Returns:
        pd.DataFrame: metrics values
    """
    # calculating metric values
    sil_score = [silhouette_score(dtw_matrix, kmeans, metric='precomputed'), 
            silhouette_score(dtw_matrix, agg, metric='precomputed'), 
            silhouette_score(dtw_matrix, dbscan, metric='precomputed')]
    dunn_index = [clValid.dunn(dtw_matrix, kmeans+1)[0], 
            clValid.dunn(dtw_matrix, agg+1)[0],
            clValid.dunn(dtw_matrix, dbscan+1)[0]]
    ch_score = [symbolicDA.index_G1d(dtw_matrix, kmeans+1)[0], 
            symbolicDA.index_G1d(dtw_matrix, agg+1)[0],
            symbolicDA.index_G1d(dtw_matrix, dbscan+1)[0]]
    results = pd.DataFrame([sil_score, dunn_index, ch_score], 
                            columns=['Kmeans', 'Hierarchical', 'DBSCAN'], index=['Silhouette', 'Dunn', 'C-H'])
    return results



In [292]:
def calculate_dtw(data: pd.DataFrame) -> np.matrix:
    # creating distance matrix for searching for optimal parameters
    # transform input data into adequate structure - 3D numpy array
    data_t = data.melt(id_vars=['countrycode','country','year'])
    data_t = data_t.groupby(['countrycode','country','year','variable'])['value'].aggregate('mean').unstack('year')
    data_t = data_t.reset_index().drop('variable', axis=1).groupby(['countrycode', 'country']).agg(list)
    n_countries = data_t.shape[0] # number of points (countries)
    time_range =  data_t.shape[1] # time range
    n_vars = data.shape[1] - 3 # number of economic indexes
    # filling the array
    data_t_arr = np.empty(shape=(n_countries, time_range, n_vars))
    for i in range(n_countries):
        for j in range(time_range):
            data_t_arr[i][j] = np.array(data_t.iloc[i,j])
    # calculating distances between points (countries)
    dtw_matrix = dtw_ndim.distance_matrix_fast(data_t_arr, n_vars)
    return dtw_matrix

def calculate_euc(data: pd.DataFrame) -> np.matrix:
    data_t = data.melt(id_vars=['countrycode','country','year'])
    data_t = data_t.groupby(['countrycode','country','year', 'variable'])['value'].aggregate('mean').unstack('variable')
    data_t = data_t.reset_index().drop('year', axis=1).groupby(['countrycode', 'country']).agg(list)
    n_countries = data_t.shape[0] # number of points (countries)
    n_vars =  data.shape[1] - 3
    time_range = len(data_t.iloc[0,0]) 
    # filling the array
    data_t_arr = np.empty(shape=(n_countries, n_vars, time_range))
    for i in range(n_countries):
        for j in range(n_vars):
            data_t_arr[i][j] = np.array(data_t.iloc[i,j])
    data_t_arr_flat = np.empty(shape=(n_countries, n_vars*time_range))
    for i in range(data_t_arr.shape[0]):
        data_t_arr_flat[i] = np.concatenate(data_t_arr[i])
    euc_matrix = euclidean_distances(data_t_arr_flat, data_t_arr_flat)
    return euc_matrix

## II. Data

In [293]:
# reading data
data = pd.read_csv('data/data.csv')
data_imp = pd.read_csv('data/data_imputed.csv')
data_box = pd.read_csv('data/data_box.csv')
data_log = pd.read_csv('data/data_log.csv')
data_out = pd.read_csv('data/data_out.csv')

In [294]:
# extracting list of pairs (country name + country code) for plots
countries = data[['countrycode','country']].drop_duplicates().reset_index(drop=True)

In [295]:
euc_matrix = calculate_euc(data)
dtw_matrix = calculate_dtw(data)
euc_matrix_box = calculate_euc(data_box)
dtw_matrix_box = calculate_dtw(data_box)
euc_matrix_log = calculate_euc(data_log)
dtw_matrix_log = calculate_dtw(data_log)
euc_matrix_out = calculate_euc(data_out)
dtw_matrix_out = calculate_dtw(data_out)

In [299]:
matrices = [euc_matrix, euc_matrix_box, euc_matrix_log, dtw_matrix, dtw_matrix_box, dtw_matrix_log]
dataframes = [data, data_box, data_log]
metrics = ['euclidean', 'dtw']
# clustering algorithms comparison
# arrays for metrics values
k_max = 8
silhouette = []
chscore = []
dunnindex = []
for m in metrics:
    for d in dataframes:
        for k in range(2, k_max+1): # KMeans
            kmeans = kmeans_clustering(d, k, m)
            silhouette.append(silhouette_score(dtw_matrix, kmeans.labels_))
            chscore.append(symbolicDA.index_G1d(dtw_matrix, kmeans.labels_+1)[0])
            dunnindex.append(clValid.dunn(dtw_matrix, kmeans.labels_+1)[0])
for m in matrices:
    for link in ['average', 'complete', 'single']: # Agglomerative (different linkages)
        for k in range(2, k_max+1):
            agg = agglomerative_clustering(m, k, linkage=link)
            silhouette.append(silhouette_score(m, agg.labels_))
            chscore.append(symbolicDA.index_G1d(m, agg.labels_+1)[0])
            dunnindex.append(clValid.dunn(m, agg.labels_+1)[0])
metrics = pd.DataFrame({'silhouette' : silhouette, 'chscore' : chscore, 'dunnindex' : dunnindex})
metrics['data'] = pd.Series(['Euc']*7 + ['Euc_box']*7 + ['Euc_log']*7 + ['Dtw']*7 + ['Dtw_box']*7 + ['Dtw_log']*7 + ['Euc']*21 + ['Euc_box']*21 + ['Euc_log']*21 + ['Dtw']*21 + ['Dtw_box']*21 + ['Dtw_log']*21)
metrics['algorithm'] = pd.Series(['KMeans']*42 + ['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7 +['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7 + ['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7 +['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7 +['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7 + ['Hierarchical average']*7 + ['Hierarchical complete']*7 + ['Hierarchical single']*7)
metrics['n_clusters'] = pd.Series([x for x in range(2,9)]*4*6)
metrics = metrics[['data', 'algorithm', 'n_clusters', 'silhouette', 'chscore', 'dunnindex']]

In [300]:
metrics['name'] = metrics['data'] + " " + metrics['algorithm']
metrics = metrics.drop('data', axis=1)
metrics['algorithm'] = metrics['name']
metrics = metrics.drop('name', axis=1)

In [301]:
metrics.tail(54)

,algorithm,n_clusters,silhouette,chscore,dunnindex
114,Dtw Hierarchical complete,4,0.310625,62.274435,0.379686
115,Dtw Hierarchical complete,5,0.309771,56.630388,0.394044
116,Dtw Hierarchical complete,6,0.270279,46.172979,0.415432
117,Dtw Hierarchical complete,7,0.252946,55.855490,0.441302
118,Dtw Hierarchical complete,8,0.229414,47.180833,0.444630
119,Dtw Hierarchical single,2,0.379560,4.298158,0.511341
120,Dtw Hierarchical single,3,0.310679,4.298448,0.472066
121,Dtw Hierarchical single,4,0.043691,3.875448,0.442141
122,Dtw Hierarchical single,5,0.088885,5.819122,0.478705
123,Dtw Hierarchical single,6,0.045140,4.533649,0.477780


In [304]:
idx = metrics.groupby(['algorithm'])['silhouette'].transform(max) == metrics['silhouette']
metrics[idx]

,algorithm,n_clusters,silhouette,chscore,dunnindex
1,Euc KMeans,3,0.288969,52.839357,0.293818
7,Euc_box KMeans,2,0.090977,48.268526,0.176115
17,Euc_log KMeans,5,0.285915,47.432653,0.379686
22,Dtw KMeans,3,0.288969,52.839357,0.293818
28,Dtw_box KMeans,2,0.090977,48.268526,0.176115
35,Dtw_log KMeans,2,0.288990,64.579902,0.237079
43,Euc Hierarchical average,3,0.314123,6.570819,0.414722
51,Euc Hierarchical complete,4,0.308091,74.734342,0.363967
56,Euc Hierarchical single,2,0.379384,4.289905,0.511341
63,Euc_box Hierarchical average,2,0.487539,20.214097,0.214006


In [302]:
# initializing figure
fig = go.Figure()
buttons = list()
for i in range(metrics.shape[1]-2):
    m = metrics.columns[i+2,]
    df_test = metrics[['algorithm','n_clusters', m]]

    # transposing data
    df_test_transposed = df_test.pivot_table(index='algorithm', columns=['n_clusters'], values=m).reset_index()
    df_test_final = df_test_transposed.rename_axis('').rename_axis("", axis="columns").set_index('algorithm')

    # adding traces
    for alg in df_test_final.index:
        if i==0: # setting first layer to be visible on the load
            fig.add_trace(go.Scatter(x=df_test_final.columns, y=df_test_final.loc[alg],
                    name=alg, visible=True))            
        else:
            fig.add_trace(go.Scatter(x=df_test_final.columns, y=df_test_final.loc[alg],
                    name=alg, visible=False))
    n_of_countries = df_test_final.shape[0]
    # setting visibility
    visible = [False]*n_of_countries*i + [True]*n_of_countries + [False]*n_of_countries*(n_of_countries-i-1)
    buttons.append(dict(label = m,
                method = 'update',
                args = [{'visible': visible},
                        {'title': m}]))    
fig.update_layout(dict(updatemenus=[dict(
    type='dropdown', buttons=buttons, xanchor='right', x=1, y=1.15, active=0)],
    title='Metrics'))
# saving plot to HTML file
plot(fig, filename='metrics.html')

'metrics.html'